<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [19]:
import pandas as pd
link_test = 'https://raw.githubusercontent.com/sumandutta8877/DATA-Analytics/16ba5c4c644c6c41d4c70404bd058df8b41979cd/Data/Test_data.csv?token=AKJPV77XCMAD2DFDGOWNM5DAIXSIC'
link_train = 'https://raw.githubusercontent.com/sumandutta8877/DATA-Analytics/16ba5c4c644c6c41d4c70404bd058df8b41979cd/Data/Train_data.csv?token=AKJPV7ZL2KMTXBEJF2VN7YLAIXSOS'

In [20]:
df_test = pd.read_csv(link_test)
print(df_test)
df_train = pd.read_csv(link_train)
print(df_train)

HTTPError: ignored

# Start Processing

In [ ]:
print('Hello Suman\'s World!')
print('Load CSV File First')

Hello Suman's World!
Load CSV File First


In [ ]:
print(df1)

         id  acousticness  danceability  ...  valence  year  duration-min
0      4943       0.03360         0.523  ...    0.346  2014           3.8
1     16165       0.76500         0.687  ...    0.830  1972           3.8
2      6774       0.36300         0.536  ...    0.289  1998           4.4
3      9959       0.00463         0.890  ...    0.970  1980           3.9
4     13818       0.00187         0.555  ...    0.882  2007           3.2
...     ...           ...           ...  ...      ...   ...           ...
3995   6367       0.00019         0.272  ...    0.564  1981           3.5
3996  12061       0.10600         0.444  ...    0.747  2017           3.1
3997  15802       0.64900         0.556  ...    0.604  1994           3.3
3998  11746       0.99500         0.378  ...    0.158  1946           2.6
3999   8441       0.02620         0.534  ...    0.145  2017           3.9

[4000 rows x 16 columns]


In [ ]:
print(df2)

          id  acousticness  danceability  ...  year duration-min  popularity
0       2015        0.9490        0.2350  ...  1947          3.0    very low
1      15901        0.8550        0.4560  ...  2020          2.4         low
2       9002        0.8270        0.4950  ...  1950          2.6    very low
3       6734        0.6540        0.6430  ...  1974          2.4         low
4      15563        0.7380        0.7050  ...  1973          3.4     average
...      ...           ...           ...  ...   ...          ...         ...
12222  15343        0.0408        0.8090  ...  2014          3.4     average
12223   1701        0.9120        0.4510  ...  1959          2.0   very high
12224   3351        0.3280        0.5510  ...  1984          2.5         low
12225   8879        0.1220        0.0608  ...  2017          3.1        high
12226   9711        0.0380        0.3890  ...  2020          3.1        high

[12227 rows x 17 columns]
